[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/Retrieval-based-Voice-Conversion-WebUI-colab/blob/main/test1.ipynb)

In [ ]:
!apt-get -y install build-essential python3-dev ffmpeg
!pip3 install --upgrade setuptools wheel
!pip3 install --upgrade pip
!pip3 install faiss-gpu fairseq gradio ffmpeg ffmpeg-python praat-parselmouth pyworld numpy==1.23.5 numba==0.56.4 librosa==0.9.2
!apt -y install -qq aria2
!pip install mega.py --quiet
!pip install gdown --quiet

%cd /content/
!wget https://github.com/RVC-Project/Retrieval-based-Voice-Conversion-WebUI/releases/download/20230428updated/updated20230428.zip
!mkdir /content/Retrieval-based-Voice-Conversion-WebUI
!unzip -d /content/Retrieval-based-Voice-Conversion-WebUI -b updated20230428.zip
!rm updated20230428.zip
%cd /content/Retrieval-based-Voice-Conversion-WebUI
!wget https://raw.githubusercontent.com/777gt/EVC/main/easy-infer.py
!mkdir -p pretrained uvr5_weights

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/hubert_base.pt -d /content/Retrieval-based-Voice-Conversion-WebUI -o hubert_base.pt

from mega import Mega
import os
MODEL = 'BENEE'
MODELEPOCH = ''
url='https://drive.google.com/file/d/1nuUVTqg1_lja7JFctjd18W9hQ7rk7aDZ/view?usp=share_link'
MODELZIP = MODEL + '.zip'
modelname_path='/content/zips/'+MODELZIP
!mkdir -p /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODEL}
if url != '':
  !mkdir -p /content/zips/
  if "drive.google.com" in url:
    !gdown $url --fuzzy -O $modelname_path
  elif "mega.nz" in url:
    m = Mega()
    m.download_url(url, '/content/zips')
  else:
    !wget $url -O /content/zips/{MODELZIP}
  !unzip -d /content/unzips/ -B /content/zips/{MODEL}.zip
else:
  raise Exception('The URL cannot be empty.')

for root, dirs, files in os.walk('/content/unzips'):
  for file in files:
    if "G_" in file:
        MODELEPOCH = file.split("G_")[1].split(".")[0]
  if MODELEPOCH == '':
    MODELEPOCH = '404'
  for file in files:
    file_path = os.path.join(root, file)
    if file.endswith(".npy") or file.endswith(".index"):
      !mv {file_path} /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODEL}/
    elif "G_" not in file and "D_" not in file and file.endswith(".pth"):
      !mv {file_path} /content/Retrieval-based-Voice-Conversion-WebUI/weights/{MODEL}.pth

!rm -r /content/unzips/
!rm -r /content/zips/

%cd /content/Retrieval-based-Voice-Conversion-WebUI
!python3 easy-infer.py --colab --pycmd python3